In [191]:
# The code is about simulating a call center, with two kinds of customers and three kind of lines.
# MohammadMahdi Ghasemloo, Mohammad rfan Naderloo
# Last edit: 11 pm 8/5/2022


# The shift time has changes in line 164
import random
import math
import pandas as pd
import numpy as np
# Initialization
num_of_beginners = 22
num_of_specialists = 9
num_of_technicals =  8

warmup_point = 0
def Exponential(lambd):
    r = random.random()
    return -(1 / lambd) * math.log(r)


def Uniform(a, b):
    r = random.random()
    return a + (b - a) * r

# if you are not about to have so many replications and preferably one replication, uncomment the code below to have information printed.
def print_header():
    print('Event Type'.ljust(15) + '\t' + 'Event Time'.ljust(15) + '\t' +
          'Special Customer Queue Length'.ljust(15) + '\t' + 'Normal Customer Queue Length'.ljust(
        15) + '\t'   + 'Technical Special Customer Queue Length'.ljust(
        15) + '\t' + 'Technical Normal Customer Queue Length'.ljust(15) + '\t' + 'Specialist Server Status'.ljust(
        15) + '\t' + 'Beginner Server Status'.ljust(15) + 'Technical Server Status'.ljust(15))

    print('----------------------------------------------------------------------------------')


# for printing the table
def nice_print(current_state, current_event):
    print(str(current_event['Event Type']).ljust(15) + '\t' + str(round(current_event['Event Time'], 4)).ljust(
        15) + '\t' + str(current_state['Special Customer Queue Length']).ljust(15) + '\t' + str(
        current_state['Normal Customer Queue Length']).ljust(15)  + '\t' + str(
        current_state['Technical Special Customer Queue Length']).ljust(15) + '\t' + str(
        current_state['Technical Normal Customer Queue Length']).ljust(15) + '\t' + str(
        current_state['Specialist Server Status']).ljust(15) + '\t' + str(
        current_state['Beginner Server Status']).ljust(15) + '\t' + str(current_state['Technical Server Status']).ljust(
        15))
state_stored_for_monitoring = list()
state_time_monitoring = list()
state_stored = list()
state_stored_dict = dict()
# to store data from replications we define lists here.
customer_churn_total = list()

special_service_starters_total = list()
max_special_customer_queue_length_total = list()
max_normal_customer_queue_length_total = list()
max_special_customer_technical_queue_length_total = list()
max_normal_customer_technical_queue_length_total = list()
mean_special_customer_queue_length_total = list()
mean_normal_customer_queue_length_total = list()
mean_special_customer_technical_queue_length_total = list()
mean_normal_customer_technical_queue_length_total = list()

max_special_customer_queue_waiting_time_total = list()
max_normal_customer_queue_length_waiting_time_total = list()
max_special_customer_technical_queue_waiting_time_total = list()
max_normal_customer_technical_queue_waiting_time_total = list()
mean_special_customer_queue_waiting_time_total = list()
mean_normal_customer_queue_waiting_time_total = list()

mean_special_customer_technical_queue_waiting_time_total = list()
mean_normal_customer_technical_queue_waiting_time_total = list()
mean_special_customer_waiting_time_in_system_total = list()
mean_normal_customer_waiting_time_in_system_total = list()
#Yakhoda

specialist_servers_utilization_total = list()
beginner_servers_utilization_total = list()
technical_servers_utilization_total = list()

#Yakhoda
specialist_servers_overwork_total = list()
beginner_servers_overwork_total = list()
technical_servers_overwork_total = list()

# starting state function, does sum initial work like definig variables and also initializes our variables.
def starting_state():

    # State variables
    state = dict()
    state['Special Customer Queue Length'] = 0 # To store length of special customers in the very first queue
    state['Normal Customer Queue Length'] = 0 # To store length of normal customers in the very first queue
    state['Technical Special Customer Queue Length'] = 0 # To store length of special customers in technical queue
    state['Technical Normal Customer Queue Length'] = 0 # To store length of normal customers in technical queue
    state['Specialist Server Status'] = 0  # 0: Free, 1:one Busy, 2: two busy
    state['Beginner Server Status'] = 0 # 0: Free, 1:one Busy, 2: two busy, 3: three busy
    state['Technical Server Status'] = 0 # 0: Free, 1:one Busy, 2: two busy

    # Data: will save everything
    data = dict()
    data['Special Customers'] = dict()  # To track each special customer, saving their arrival time, time service begins, etc.
    data['Normal Customers'] = dict() # To track each normal customer, saving their arrival time, time service begins, etc.
    data['Last Time Special Customer Queue Length Changed'] = 0
    data['Last Time Normal Customer Queue Length Changed'] = 0
    data['Last Time Technical Special Customer Queue Length Changed'] = 0
    data['Last Time Technical Normal Customer Queue Length Changed'] = 0
    # To check which customer has first arrived, we store them in a dictionary and for each line in a distinct dictionary
    data['Special Queue Customers'] = dict()
    data['Normal Queue Customers'] = dict()
    data['Technical Special Queue Customers'] = dict()
    data['Technical Normal Queue Customers'] = dict()

    # Cumulative Stats
    data['Cumulative Stats'] = dict()
    
    # Stores last time time each operator group serviced
    data['Cumulative Stats']['Specialist Servers Last Time'] = 0
    data['Cumulative Stats']['Beginner Servers Last Time'] = 0
    data['Cumulative Stats']['Technical Servers Last Time'] = 0
    
    
    # Stores busy time in order to calculate utilization
    data['Cumulative Stats']['Specialist Servers Busy Time'] = 0
    data['Cumulative Stats']['Beginner Servers Busy Time'] = 0
    data['Cumulative Stats']['Technical Servers Busy Time'] = 0
    # Stores waiting times, each line and each customer distinctly
    data['Cumulative Stats']['Special Queue Waiting Time'] = 0
    data['Cumulative Stats']['Normal Queue Waiting Time'] = 0
    data['Cumulative Stats']['Technical Special Queue Waiting Time'] = 0
    data['Cumulative Stats']['Technical Normal Queue Waiting Time'] = 0
    # Stores area under queue length, each line and each customer distinctly
    data['Cumulative Stats']['Area Under Special Queue Length Curve'] = 0
    data['Cumulative Stats']['Area Under Normal Queue Length Curve'] = 0
    data['Cumulative Stats']['Area Under Technical Special Queue Length Curve'] = 0
    data['Cumulative Stats']['Area Under Technical Normal Queue Length Curve'] = 0
    # stores immediate service tarters in each line. At the end we implement union on these sets to get general srvice starters.
    data['Special Service Starters'] = set()
    data['Special Service Starters Technical'] = set()
    # Stores customers who left the system and probably left us.
    data['Cumulative Stats']['Customer Churn'] = 0
    # Stores maximums legnths, each line and each kind distinctly
    data['Cumulative Stats']['Max Special Customer Queue Length'] = 0
    data['Cumulative Stats']['Max Normal Customer Queue Length'] = 0
    data['Cumulative Stats']['Max Technical Special Customer Queue Length'] = 0
    data['Cumulative Stats']['Max Technical Normal Customer Queue Length'] = 0
    data['Cumulative Stats']['Max Technical Normal Customer Queue Length'] = 0
    # Stores customers of each line and each kind distinctly
    data['Special Queue Customers All'] = set()
    data['Normal Queue Customers All'] = set()
    data['Technical Special Queue Customers All'] = set()
    data['Technical Normal Queue Customers All'] = set()
    data['Special Customers Waiting Time In System'] = 0
    data['Normal Customers Waiting Time In System'] = 0
    #Yakhoda

    # Stores maximums waiting times, each line and each kind distinctly
    data['Cumulative Stats']['Max Special Customer Queue Waiting Time'] = 0
    data['Cumulative Stats']['Max Normal Customer Queue Waiting Time'] = 0
    data['Cumulative Stats']['Max Technical Special Customer Queue Waiting Time'] = 0
    data['Cumulative Stats']['Max Technical Normal Customer Queue Waiting Time'] = 0


    # Starting FEL
    future_event_list = list()
    if Uniform(0, 1) > 0.7:
        future_event_list.append(
            {'Event Type': 'Customer Call', 'Event Time': 0, 'Special Customer': 'S1', 'Normal Customer': 'C0',
             'Last Customer': 'S'})
    else:
        future_event_list.append(
            {'Event Type': 'Customer Call', 'Event Time': 0, 'Special Customer': 'S0', 'Normal Customer': 'C1',
             'Last Customer': 'C'})

    return state, future_event_list, data,

# we use this function to make our events with theirs specific features.
def fel_maker(future_event_list, event_type, state, data, clock, customer_special=None,
              customer_normal=None,
              last_customer=None):
    event_time = 0

    if event_type == 'Customer Call':
        if clock % 1440 < 16 * 60: # I changed it here!!!!!!! #yakhoda
            shift = 1
        elif clock % 1440 < 16 * 60:
            shift = 2
        else:
            shift = 3

        if shift == 1:
            event_time = clock + Exponential(1 / 0.15) #ASAP
        elif shift == 2:
            event_time = clock + Exponential(1 / 1.2) #ASAP
        else:
            event_time = clock + Exponential(1 / 1) #ASAP
    elif event_type == 'Departure From Expert':
        event_time = clock + Exponential(1 / 3) #ASAP
    elif event_type == 'Departure From Beginner':
        event_time = clock + Exponential(1 / 7) #ASAP
    elif event_type == 'Departure From Technical':
        event_time = clock + Exponential(1/10) #ASAP
    elif event_type == 'Leaving Of Tired':

        event_time = clock + Uniform(0.5, 3) #ASAP

    # According to the information given, future event will be made above and will be added to fel as follows.
    new_event = {'Event Type': event_type, 'Event Time': event_time, 'Special Customer': customer_special,
                 'Normal Customer': customer_normal, 'Last Customer': last_customer}
    future_event_list.append(new_event)


def Customer_Call(future_event_list, state, clock, data, customer_special=None, customer_normal=None,
                  last_customer=None):
    # we make the next arrival here.
    if Uniform(0, 1) > 0.6: #ASAP (Just Check if you can change this!)
        next_customer = 'S' + str(int(customer_special[1:]) + 1)
        fel_maker(future_event_list, 'Customer Call', state, data, clock,
                  customer_special=next_customer,
                  customer_normal=customer_normal, last_customer='S')

    else:
        next_customer = 'C' + str(int(customer_normal[1:]) + 1)
        fel_maker(future_event_list, 'Customer Call', state, data, clock,
                  customer_special=customer_special,
                  customer_normal=next_customer, last_customer='C')

    # This part is for the customer who has just arrived.
    if last_customer == 'S':
        # Our customer is special
        data['Special Customers'][customer_special] = dict()
        data['Special Customers'][customer_special]['Arrival Time'] = clock
        data['Special Queue Customers All'].add(customer_special)
        if state['Specialist Server Status'] < num_of_specialists: #ASAP_special
            state['Specialist Server Status'] += 1
            fel_maker(future_event_list, 'Departure From Expert', state, data, clock,
                      customer_special=customer_special, customer_normal=customer_normal, last_customer='S')
            data['Special Customers'][customer_special]['Time Service Begins'] = clock
            data['Special Service Starters'].add(customer_special)
        else:
            data['Cumulative Stats']['Area Under Special Queue Length Curve'] += state[
                                                                                     'Special Customer Queue Length'] * (
                                                                                         clock - data[
                                                                                     'Last Time Special Customer Queue Length Changed'])
            state['Special Customer Queue Length'] += 1
            if data['Cumulative Stats']['Max Special Customer Queue Length'] < state[
                'Special Customer Queue Length']:
                data['Cumulative Stats']['Max Special Customer Queue Length'] = state[
                    'Special Customer Queue Length']
            data['Special Queue Customers'][customer_special] = clock
            data['Special Queue Customers All'].add(customer_special)
            data['Last Time Special Customer Queue Length Changed'] = clock
            # will he get tired?
            if Uniform(0, 1) >= 0.85:
                # he will get tired
                fel_maker(future_event_list, 'Leaving Of Tired', state, data, clock,
                          customer_special=customer_special, customer_normal=customer_normal, last_customer='S')

    elif last_customer == "C":
        data['Normal Customers'][customer_normal] = dict()
        data['Normal Customers'][customer_normal]['Arrival Time'] = clock
        data['Normal Queue Customers All'].add(customer_normal)
        if state['Beginner Server Status'] < num_of_beginners: #ASAP_beginner
            state['Beginner Server Status'] += 1
            fel_maker(future_event_list, 'Departure From Beginner', state, data, clock,
                      customer_special=customer_special, customer_normal=customer_normal, last_customer='C')
            data['Normal Customers'][customer_normal]['Time Service Begins'] = clock
        elif state['Specialist Server Status'] < num_of_specialists: #ASAP_special
            state['Specialist Server Status'] += 1
            fel_maker(future_event_list, 'Departure From Expert', state, data, clock,
                      customer_special=customer_special, customer_normal=customer_normal, last_customer='C')
            data['Normal Customers'][customer_normal]['Time Service Begins'] = clock
        else:
            data['Cumulative Stats']['Area Under Normal Queue Length Curve'] += state[
                                                                                    'Normal Customer Queue Length'] * (
                                                                                        clock - data[
                                                                                    'Last Time Normal Customer Queue Length Changed'])
            state['Normal Customer Queue Length'] += 1
            if data['Cumulative Stats']['Max Normal Customer Queue Length'] < state['Normal Customer Queue Length']:
                data['Cumulative Stats']['Max Normal Customer Queue Length'] = state['Normal Customer Queue Length']
            data['Normal Queue Customers'][customer_normal] = clock
            data['Normal Queue Customers All'].add(customer_normal)
            data['Last Time Normal Customer Queue Length Changed'] = clock
            # will he get tired?
            if Uniform(0, 1) >= 0.85:
                # he will get tired
                fel_maker(future_event_list, 'Leaving Of Tired', state, data, clock,
                          customer_special=customer_special, customer_normal=customer_normal, last_customer='C')


def Departure_From_Expert(future_event_list, state, clock, data, customer_special=None,
                          customer_normal=None,
                          last_customer=None):
    # what is shift number?
    shift = ((clock % 1440) // 480) + 1
    
    data['Cumulative Stats']['Specialist Servers Last Time'] = clock #yakhoda
    # was it a special or normal customer?
    if last_customer == 'S':
        data['Cumulative Stats']['Specialist Servers Busy Time'] += clock - data['Special Customers'][customer_special][
            'Time Service Begins']
        # data['Special Customers'].pop(customer_special, None)
        # will he use technical service?
        data['Special Customers Waiting Time In System'] += clock - data['Special Customers'][customer_special][
            'Arrival Time']
        if Uniform(0, 1) > 0.85:
            # he will use technical service
            data['Special Customers'][customer_special]['Arrival Time Technical'] = clock
            data['Technical Special Queue Customers All'].add(customer_special)
            if state['Technical Server Status'] < num_of_technicals: #ASAP_technical
                # technical service begins immediately
                state['Technical Server Status'] += 1
                fel_maker(future_event_list, 'Departure From Technical', state, data, clock,
                          customer_special=customer_special, customer_normal=customer_normal, last_customer='S')
                data['Special Customers'][customer_special]['Time Service Begins Technical'] = clock
                data['Special Service Starters Technical'].add(customer_special)
            else:
                # gets in line in technical service
                data['Cumulative Stats']['Area Under Technical Special Queue Length Curve'] += state[
                                                                                                   'Technical Special Customer Queue Length'] * (
                                                                                                       clock - data[
                                                                                                   'Last Time Technical Special Customer Queue Length Changed'])
                state['Technical Special Customer Queue Length'] += 1
                if data['Cumulative Stats']['Max Technical Special Customer Queue Length'] < state[
                    'Technical Special Customer Queue Length']:
                    data['Cumulative Stats']['Max Technical Special Customer Queue Length'] = state[
                        'Technical Special Customer Queue Length']
                data['Technical Special Queue Customers'][customer_special] = clock
                data['Technical Special Queue Customers All'].add(customer_special)
                data['Last Time Technical Special Customer Queue Length Changed'] = clock
    else:
        # customer is normal
        data['Cumulative Stats']['Specialist Servers Busy Time'] += clock - data['Normal Customers'][customer_normal][
            'Time Service Begins']
        
        data['Normal Customers Waiting Time In System'] += clock - data['Normal Customers'][customer_normal][
            'Arrival Time']
        #Yakhoda
        data['Normal Customers'][customer_normal]['Time Service Ends'] = clock
        # will he use technical service?
        if Uniform(0, 1) > 0.85:
            # he will use technical service
            data['Normal Customers'][customer_normal]['Arrival Time Technical'] = clock
            data['Technical Normal Queue Customers All'].add(customer_normal)
            if state['Technical Server Status'] < num_of_technicals: #ASAP_technical
                # technical service begins immediately
                state['Technical Server Status'] += 1
                fel_maker(future_event_list, 'Departure From Technical', state, data, clock,
                          customer_special=customer_special, customer_normal=customer_normal, last_customer='C')
                data['Normal Customers'][customer_normal]['Time Service Begins Technical'] = clock
            else:
                # gets in line in technical service
                data['Cumulative Stats']['Area Under Technical Normal Queue Length Curve'] += state[
                                                                                                  'Technical Normal Customer Queue Length'] * (
                                                                                                      clock - data[
                                                                                                  'Last Time Technical Normal Customer Queue Length Changed'])
                state['Technical Normal Customer Queue Length'] += 1
                if data['Cumulative Stats']['Max Technical Normal Customer Queue Length'] < state[
                    'Technical Normal Customer Queue Length']:
                    data['Cumulative Stats']['Max Technical Normal Customer Queue Length'] = state[
                        'Technical Normal Customer Queue Length']
                data['Technical Normal Queue Customers'][customer_normal] = clock
                data['Technical Normal Queue Customers All'].add(customer_normal)
                data['Last Time Technical Normal Customer Queue Length Changed'] = clock

    # Now after dealing with the customer woh left, wh go further checking the lines.
    if state['Special Customer Queue Length'] > 0:
        # who's the first in line?
        first_customer_in_queue = min(data['Special Queue Customers'], key=data['Special Queue Customers'].get)
        # this customer starts getting service
        data['Special Customers'][first_customer_in_queue]['Time Service Begins'] = clock
        # Update queue waiting time
        data['Cumulative Stats']['Special Queue Waiting Time'] += \
            clock - data['Special Customers'][first_customer_in_queue]['Arrival Time']
        if data['Cumulative Stats']['Max Special Customer Queue Waiting Time'] < clock - \
                data['Special Customers'][first_customer_in_queue]['Arrival Time']:
            data['Cumulative Stats']['Max Special Customer Queue Waiting Time'] = clock - data['Special Customers'][
                first_customer_in_queue]['Arrival Time']
        # Queue length changes, so calculate the area under the current rectangle
        data['Cumulative Stats']['Area Under Special Queue Length Curve'] += \
            state['Special Customer Queue Length'] * (clock - data['Last Time Special Customer Queue Length Changed'])
        state['Special Customer Queue Length'] -= 1
        # This customer no longer belongs to queue
        data['Special Queue Customers'].pop(first_customer_in_queue, None)
        # Queue length just changed. We should update it.
        data['Last Time Special Queue Length Changed'] = clock
        # Schedule its specific 'End of Service'.
        fel_maker(future_event_list, 'Departure From Expert', state, data, clock,
                  customer_special=first_customer_in_queue,
                  customer_normal=customer_normal, last_customer='S')

    elif state['Normal Customer Queue Length'] > 0:
        # who's the first in line?
        first_customer_in_queue = min(data['Normal Queue Customers'], key=data['Normal Queue Customers'].get)
        # this customer starts getting service
        data['Normal Customers'][first_customer_in_queue]['Time Service Begins'] = clock
        # Update queue waiting time
        data['Cumulative Stats']['Normal Queue Waiting Time'] += \
            clock - data['Normal Customers'][first_customer_in_queue]['Arrival Time']
        if data['Cumulative Stats']['Max Normal Customer Queue Waiting Time'] < clock - \
                data['Normal Customers'][first_customer_in_queue]['Arrival Time']:
            data['Cumulative Stats']['Max Normal Customer Queue Waiting Time'] = clock - \
                                                                                 data['Normal Customers'][
                                                                                     first_customer_in_queue][
                                                                                     'Arrival Time']

        # Queue length changes, so calculate the area under the current rectangle
        data['Cumulative Stats']['Area Under Normal Queue Length Curve'] += \
            state['Normal Customer Queue Length'] * (clock - data['Last Time Normal Customer Queue Length Changed'])
        # Logic
        state['Normal Customer Queue Length'] -= 1
        # This customer no longer belongs to queue
        data['Normal Queue Customers'].pop(first_customer_in_queue, None)
        # Queue length just changed. Update 'Last Time Queue Length Changed'
        data['Last Time Normal Queue Length Changed'] = clock
        # Schedule 'End of Service' for this customer
        fel_maker(future_event_list, 'Departure From Expert', state, data, clock,
                  customer_special=customer_special,
                  customer_normal=first_customer_in_queue, last_customer='C')

    else:
        # nothing to do
        state['Specialist Server Status'] -= 1


def Departure_From_Beginner(future_event_list, state, clock, data, customer_special=None,
                            customer_normal=None,
                            last_customer=None):
    # what is shift number
    shift = ((clock % 1440) // 480) + 1
    data['Cumulative Stats']['Beginner Servers Last Time'] = clock
    data['Cumulative Stats']['Beginner Servers Busy Time'] += clock - data['Normal Customers'][customer_normal][
        'Time Service Begins']
    data['Normal Customers Waiting Time In System'] += clock - data['Normal Customers'][customer_normal][
            'Arrival Time']
    #Yakhoda
    data['Normal Customers'][customer_normal]['Time Service Ends'] = clock

    # will he use technical service?
    if Uniform(0, 1) > 0.85:
        # he will use technical service
        data['Normal Customers'][customer_normal]['Arrival Time Technical'] = clock
        data['Technical Normal Queue Customers All'].add(customer_normal)
        if state['Technical Server Status'] < num_of_technicals:#ASAP_techincal
            # technical service begins immediately
            state['Technical Server Status'] += 1
            fel_maker(future_event_list, 'Departure From Technical', state, data, clock,
                      customer_special=customer_special, customer_normal=customer_normal, last_customer='C')
            data['Normal Customers'][customer_normal]['Time Service Begins Technical'] = clock
        else:
            # gets in line in technical service
            data['Cumulative Stats']['Area Under Technical Normal Queue Length Curve'] += state[
                                                                                              'Technical Normal Customer Queue Length'] * (
                                                                                                  clock - data[
                                                                                              'Last Time Technical Normal Customer Queue Length Changed'])
            state['Technical Normal Customer Queue Length'] += 1
            if data['Cumulative Stats']['Max Technical Normal Customer Queue Length'] < state[
                'Technical Normal Customer Queue Length']:
                data['Cumulative Stats']['Max Technical Normal Customer Queue Length'] = state[
                    'Technical Normal Customer Queue Length']
            data['Technical Normal Queue Customers'][customer_normal] = clock
            data['Technical Normal Queue Customers All'].add(customer_normal)
            data['Last Time Technical Normal Customer Queue Length Changed'] = clock

    # Now we go to check the lines and other stuff
    if state['Normal Customer Queue Length'] > 0:
        # who's the first in line?
        first_customer_in_queue = min(data['Normal Queue Customers'], key=data['Normal Queue Customers'].get)
        # this customer starts getting service
        data['Normal Customers'][first_customer_in_queue]['Time Service Begins'] = clock
        # Update queue waiting time
        data['Cumulative Stats']['Normal Queue Waiting Time'] += \
            clock - data['Normal Customers'][first_customer_in_queue]['Arrival Time']
        if data['Cumulative Stats']['Max Normal Customer Queue Waiting Time'] < clock - \
                data['Normal Customers'][first_customer_in_queue]['Arrival Time']:
            data['Cumulative Stats']['Max Normal Customer Queue Waiting Time'] = clock - \
                                                                                 data['Normal Customers'][
                                                                                     first_customer_in_queue][
                                                                                     'Arrival Time']
        # Queue length changes, so calculate the area under the current rectangle
        data['Cumulative Stats']['Area Under Normal Queue Length Curve'] += \
            state['Normal Customer Queue Length'] * (clock - data['Last Time Normal Customer Queue Length Changed'])
        state['Normal Customer Queue Length'] -= 1
        # This customer no longer belongs to queue
        data['Normal Queue Customers'].pop(first_customer_in_queue, None)
        # Queue length just changed. Update 'Last Time Queue Length Changed'
        data['Last Time Normal Queue Length Changed'] = clock
        # Schedule its specific 'End of Service' for this customer
        fel_maker(future_event_list, 'Departure From Beginner', state, data, clock,
                  customer_special=customer_special,
                  customer_normal=first_customer_in_queue, last_customer='C')

    else:
        state['Beginner Server Status'] -= 1


def Departure_From_Technical(future_event_list, state, clock, data, customer_special=None,
                             customer_normal=None,
                             last_customer=None):
    data['Cumulative Stats']['Technical Servers Last Time'] = clock
    if last_customer == 'S':
        data['Cumulative Stats']['Technical Servers Busy Time'] += clock - data['Special Customers'][customer_special][
            'Time Service Begins Technical']
        data['Special Customers Waiting Time In System'] += clock - data['Special Customers'][customer_special][
            'Arrival Time Technical']
    else:
        # customer is normal
        data['Cumulative Stats']['Technical Servers Busy Time'] += clock - data['Normal Customers'][customer_normal][
            'Time Service Begins Technical']
        data['Normal Customers Waiting Time In System'] += clock - data['Normal Customers'][customer_normal][
            'Arrival Time Technical']
        #Yakhoda
    # now we go check the line
    if state['Technical Special Customer Queue Length'] > 0:
        # who's the first in line?
        first_customer_in_queue = min(data['Technical Special Queue Customers'],
                                      key=data['Technical Special Queue Customers'].get)
        # this customer starts getting service
        data['Special Customers'][first_customer_in_queue]['Time Service Begins Technical'] = clock
        # Update queue waiting time
        data['Cumulative Stats']['Technical Special Queue Waiting Time'] += \
            clock - data['Special Customers'][first_customer_in_queue]['Arrival Time Technical']
        if data['Cumulative Stats']['Max Technical Special Customer Queue Waiting Time'] < clock - \
                data['Special Customers'][first_customer_in_queue]['Arrival Time Technical']:
            data['Cumulative Stats']['Max Technical Special Customer Queue Waiting Time'] = clock - \
                                                                                            data['Special Customers'][
                                                                                                first_customer_in_queue][
                                                                                                'Arrival Time Technical']
        # Queue length changes, so calculate the area under the current rectangle
        data['Cumulative Stats']['Area Under Technical Special Queue Length Curve'] += \
            state['Technical Special Customer Queue Length'] * (
                    clock - data['Last Time Technical Special Customer Queue Length Changed'])
        # Logic
        state['Technical Special Customer Queue Length'] -= 1
        # This customer no longer belongs to queue
        data['Technical Special Queue Customers'].pop(first_customer_in_queue, None)
        # Queue length just changed. Update 'Last Time Queue Length Changed'
        data['Last Time Technical Special Customer Queue Length Changed'] = clock
        # Schedule its specific 'End of Service' for this customer
        fel_maker(future_event_list, 'Departure From Technical', state, data, clock,
                  customer_special=first_customer_in_queue,
                  customer_normal=customer_normal, last_customer='S')

    elif state['Technical Normal Customer Queue Length'] > 0:
        # who's the first in line?
        first_customer_in_queue = min(data['Technical Normal Queue Customers'],
                                      key=data['Technical Normal Queue Customers'].get)
        # this customer starts getting service
        data['Normal Customers'][first_customer_in_queue]['Time Service Begins Technical'] = clock
        # Update queue waiting time
        data['Cumulative Stats']['Technical Normal Queue Waiting Time'] += \
            clock - data['Normal Customers'][first_customer_in_queue]['Arrival Time Technical']
        if data['Cumulative Stats']['Max Technical Normal Customer Queue Waiting Time'] < clock - \
                data['Normal Customers'][first_customer_in_queue]['Arrival Time Technical']:
            data['Cumulative Stats']['Max Technical Normal Customer Queue Waiting Time'] = clock - \
                                                                                           data['Normal Customers'][
                                                                                               first_customer_in_queue][
                                                                                               'Arrival Time Technical']
        # Queue length changes, so calculate the area under the current rectangle
        data['Cumulative Stats']['Area Under Technical Normal Queue Length Curve'] += \
            state['Technical Normal Customer Queue Length'] * (
                    clock - data['Last Time Technical Normal Customer Queue Length Changed'])
        # Logic
        state['Technical Normal Customer Queue Length'] -= 1
        # This customer no longer belongs to queue
        data['Technical Normal Queue Customers'].pop(first_customer_in_queue, None)
        # Queue length just changed. Update 'Last Time Queue Length Changed'
        data['Last Time Technical Normal Customer Queue Length Changed'] = clock
        # Schedule its 'End of Service' for this customer
        fel_maker(future_event_list, 'Departure From Technical', state, data, clock,
                  customer_special=customer_special,
                  customer_normal=first_customer_in_queue, last_customer='C')
    else:
        state['Technical Server Status'] -= 1


def Leaving_Of_Tired(future_event_list, state, clock, data, customer_special=None, customer_normal=None,
                     last_customer=None):
    if last_customer == 'S':
        data['Special Customers Waiting Time In System'] += clock - data['Special Customers'][
            customer_special]['Arrival Time']
        # customer is special
        # here we check if service for this customer has begun
        if 'Time Service Begins' in data['Special Customers'][customer_special].keys():
            # The Service has begun and we do nothing if so.
            True
        else:
            # this customer is one that leaves
            # Update queue waiting time
            data['Cumulative Stats']['Special Queue Waiting Time'] += \
                clock - data['Special Customers'][customer_special]['Arrival Time']
            if data['Cumulative Stats']['Max Special Customer Queue Waiting Time'] < clock - \
                    data['Special Customers'][customer_special]['Arrival Time']:
                data['Cumulative Stats']['Max Special Customer Queue Waiting Time'] = clock - data['Special Customers'][
                    customer_special]['Arrival Time']
            # Queue length changes, so calculate the area under the current rectangle
            data['Cumulative Stats']['Area Under Special Queue Length Curve'] += \
                state['Special Customer Queue Length'] * (
                        clock - data['Last Time Special Customer Queue Length Changed'])
            # Logic
            state['Special Customer Queue Length'] -= 1
            # This customer no longer belongs to queue
            data['Special Queue Customers'].pop(customer_special, None)
            # Queue length just changed. Update 'Last Time Queue Length Changed'
            data['Last Time Special Queue Length Changed'] = clock
            data['Cumulative Stats']['Customer Churn'] += 1
            data['Special Customers'][customer_special]['Leave Time'] = clock


    else:
        # customer is normal
        # here we check if service for this customer has begun
        if 'Time Service Begins' in data['Normal Customers'][customer_normal].keys():
            # The Service has begun and we do nothing if so.
            True
        else:
            # this customer is one that leaves
            # Update queue waiting time
            data['Cumulative Stats']['Normal Queue Waiting Time'] += \
                clock - data['Normal Customers'][customer_normal]['Arrival Time']
            if data['Cumulative Stats']['Max Normal Customer Queue Waiting Time'] < clock - \
                    data['Normal Customers'][customer_normal]['Arrival Time']:
                data['Cumulative Stats']['Max Normal Customer Queue Waiting Time'] = clock - \
                                                                                     data['Normal Customers'][
                                                                                         customer_normal][
                                                                                         'Arrival Time']
            # Queue length changes, so calculate the area under the current rectangle
            data['Cumulative Stats']['Area Under Normal Queue Length Curve'] += \
                state['Normal Customer Queue Length'] * (
                        clock - data['Last Time Normal Customer Queue Length Changed'])
            # Logic
            state['Normal Customer Queue Length'] -= 1
            # This customer no longer belongs to queue
            data['Normal Queue Customers'].pop(customer_normal, None)
            # Queue length just changed. Update 'Last Time Queue Length Changed'
            data['Last Time Normal Queue Length Changed'] = clock
            data['Cumulative Stats']['Customer Churn'] += 1
            data['Normal Customers'][customer_normal]['Leave Time'] = clock







def simulation(simulation_time):
    state, future_event_list, data = starting_state()
    month = 1
    clock = 0
    table = []
    step = 1
    work_time = 10*60
    future_event_list.append(
        {'Event Type': 'End of Simulation', 'Event Time': simulation_time, 'Special Customer': None,
         'Normal Customer': None, 'Last Customer': None})

    clock_old = 0
    clock_old_min = 0
    while clock < simulation_time:
        
        if int(clock/60)!= clock_old:
#             print((clock - clock_old_min))
            # Here you should first store them and then delelte them . 
            # storing 
            max_special_customer_queue_waiting_time = data['Cumulative Stats']['Max Special Customer Queue Waiting Time']
            max_normal_customer_queue_length_waiting_time = data['Cumulative Stats']['Max Normal Customer Queue Waiting Time']
            max_special_customer_technical_queue_waiting_time = data['Cumulative Stats'][
        'Max Technical Special Customer Queue Waiting Time']
            max_normal_customer_technical_queue_waiting_time = data['Cumulative Stats'][
        'Max Technical Normal Customer Queue Waiting Time']
            specialist_servers_utilization = min(data['Cumulative Stats']['Specialist Servers Busy Time'] / ((clock - clock_old_min)*num_of_specialists) , 1) # should be modified
            beginner_servers_utilization = min(data['Cumulative Stats']['Beginner Servers Busy Time'] / ((clock - clock_old_min)*num_of_beginners),1 )# should be modified
            customer_churn = data['Cumulative Stats']['Customer Churn']
            technical_servers_utilization = min(data['Cumulative Stats']['Technical Servers Busy Time']/((clock - clock_old_min)*num_of_technicals),1)
            
#             print(technical_servers_utilization)
            
            current_output_tuple = (max_special_customer_queue_waiting_time, 
                                    max_normal_customer_queue_length_waiting_time, 
                                    max_special_customer_technical_queue_waiting_time,
                                   max_normal_customer_technical_queue_waiting_time,
                                   specialist_servers_utilization,
                                    beginner_servers_utilization,
                                    customer_churn, 
                                    technical_servers_utilization, 
                                    
                                    
                                   )
            # deleting
            data['Cumulative Stats']['Specialist Servers Last Time'] = 0
            data['Cumulative Stats']['Beginner Servers Last Time'] = 0
            data['Cumulative Stats']['Technical Servers Last Time'] = 0


            # Stores busy time in order to calculate utilization
            data['Cumulative Stats']['Specialist Servers Busy Time'] = 0
            data['Cumulative Stats']['Beginner Servers Busy Time'] = 0
            data['Cumulative Stats']['Technical Servers Busy Time'] = 0
            # Stores waiting times, each line and each customer distinctly
            data['Cumulative Stats']['Special Queue Waiting Time'] = 0
            data['Cumulative Stats']['Normal Queue Waiting Time'] = 0
            data['Cumulative Stats']['Technical Special Queue Waiting Time'] = 0
            data['Cumulative Stats']['Technical Normal Queue Waiting Time'] = 0
            # Stores area under queue length, each line and each customer distinctly
            data['Cumulative Stats']['Area Under Special Queue Length Curve'] = 0
            data['Cumulative Stats']['Area Under Normal Queue Length Curve'] = 0
            data['Cumulative Stats']['Area Under Technical Special Queue Length Curve'] = 0
            data['Cumulative Stats']['Area Under Technical Normal Queue Length Curve'] = 0
            # stores immediate service tarters in each line. At the end we implement union on these sets to get general srvice starters.
            data['Special Service Starters'] = set()
            data['Special Service Starters Technical'] = set()
            # Stores customers who left the system and probably left us.
            data['Cumulative Stats']['Customer Churn'] = 0
            # Stores maximums legnths, each line and each kind distinctly
            data['Cumulative Stats']['Max Special Customer Queue Length'] = 0
            data['Cumulative Stats']['Max Normal Customer Queue Length'] = 0
            data['Cumulative Stats']['Max Technical Special Customer Queue Length'] = 0
            data['Cumulative Stats']['Max Technical Normal Customer Queue Length'] = 0
            data['Cumulative Stats']['Max Technical Normal Customer Queue Length'] = 0
            data['Cumulative Stats']['Max Special Customer Queue Waiting Time'] = 0
            data['Cumulative Stats']['Max Normal Customer Queue Waiting Time'] = 0
            data['Cumulative Stats']['Max Technical Special Customer Queue Waiting Time'] = 0
            data['Cumulative Stats']['Max Technical Normal Customer Queue Waiting Time'] = 0


            
        
        
            # Here check the currrent state and gather the results
            current_state_tuple = (state['Special Customer Queue Length'], state['Normal Customer Queue Length'], state['Technical Special Customer Queue Length'], state['Technical Normal Customer Queue Length'])
            state_stored.append(current_state_tuple)
            if current_state_tuple in state_stored_dict:
                state_stored_dict[current_state_tuple].append(current_output_tuple) 
                
            else:
                state_stored_dict[current_state_tuple] = list()
                state_stored_dict[current_state_tuple].append(current_output_tuple)
            
            
            clock_old = int(clock/60)
            clock_old_min = clock
        sorted_fel = sorted(future_event_list, key=lambda x: x['Event Time'])

        current_event = sorted_fel[0]
        clock = current_event['Event Time']
        customer_special = current_event['Special Customer']
        customer_normal = current_event['Normal Customer']
        last_customer = current_event['Last Customer']
        if clock < simulation_time:# yakhoda
            state_stored_for_monitoring.append((state['Special Customer Queue Length'], state['Normal Customer Queue Length'], state['Technical Special Customer Queue Length'], state['Technical Normal Customer Queue Length']))
            state_time_monitoring.append(clock)
            if current_event['Event Type'] == 'Customer Call' and  clock<work_time:     #yakhoda
                Customer_Call(future_event_list, state, clock, data,
                              customer_special=customer_special,
                              customer_normal=customer_normal, last_customer=last_customer)
            elif current_event['Event Type'] == 'Departure From Expert':
                Departure_From_Expert(future_event_list, state, clock, data,
                                      customer_special=customer_special,
                                      customer_normal=customer_normal, last_customer=last_customer)
            elif current_event['Event Type'] == 'Departure From Beginner':
                Departure_From_Beginner(future_event_list, state, clock, data,
                                        customer_special=customer_special,
                                        customer_normal=customer_normal, last_customer=last_customer)
            elif current_event['Event Type'] == 'Departure From Technical':
                Departure_From_Technical(future_event_list, state, clock, data,
                                         customer_special=customer_special,
                                         customer_normal=customer_normal, last_customer=last_customer)
            elif current_event['Event Type'] == 'Leaving Of Tired':
                Leaving_Of_Tired(future_event_list, state, clock, data,
                                 customer_special=customer_special,
                                 customer_normal=customer_normal, last_customer=last_customer)
            future_event_list.remove(current_event)
        else:
            future_event_list.clear()
        step += 1
        # nice_print(state, current_event)
        if clock < warmup_point:
            for key in data['Cumulative Stats']:
                data['Cumulative Stats'][key] = 0
        # print(clock)
        # print('**********************************')
        # print(data['Cumulative Stats'])
    print('----------------------------------------------------------------------------------')
    # # If you want to run for more than one replication, comment the first three lines of the code below otherwise you may experience a slow run.
    # excel_main_header = create_main_header(state, data)
    # justify(table)
    # create_excel(table, excel_main_header)
    print('Simulation Ended!')
    if data['Technical Special Queue Customers All'] == set():
        data['Technical Special Queue Customers All'] = {-1}
    if data['Technical Normal Queue Customers All'] == set():
        data['Technical Normal Queue Customers All'] = {-1}
    customer_churn = data['Cumulative Stats']['Customer Churn'] / (
                len(data['Special Queue Customers All']) + len(data['Normal Queue Customers All']))


    special_service_starters = (len(
        data['Special Service Starters'].intersection(data['Special Service Starters Technical']))) / (len(
        data['Special Queue Customers All']))
    max_special_customer_queue_length = data['Cumulative Stats']['Max Special Customer Queue Length']
    max_normal_customer_queue_length = data['Cumulative Stats']['Max Normal Customer Queue Length']


    max_special_customer_technical_queue_length = data['Cumulative Stats'][
        'Max Technical Special Customer Queue Length']
    max_normal_customer_technical_queue_length = data['Cumulative Stats']['Max Technical Normal Customer Queue Length']
    mean_special_customer_queue_length = data['Cumulative Stats'][
                                             'Area Under Special Queue Length Curve'] / (simulation_time - warmup_point)
    mean_normal_customer_queue_length = data['Cumulative Stats'][
                                            'Area Under Normal Queue Length Curve'] / (simulation_time - warmup_point)
    mean_special_customer_technical_queue_length = data['Cumulative Stats'][
                                                       'Area Under Technical Special Queue Length Curve'] / (simulation_time - warmup_point)
    mean_normal_customer_technical_queue_length = data['Cumulative Stats'][
                                                      'Area Under Technical Normal Queue Length Curve'] / (simulation_time - warmup_point)

    max_special_customer_queue_waiting_time = data['Cumulative Stats']['Max Special Customer Queue Waiting Time']
    max_normal_customer_queue_length_waiting_time = data['Cumulative Stats']['Max Normal Customer Queue Waiting Time']

    max_special_customer_technical_queue_waiting_time = data['Cumulative Stats'][
        'Max Technical Special Customer Queue Waiting Time']
    max_normal_customer_technical_queue_waiting_time = data['Cumulative Stats'][
        'Max Technical Normal Customer Queue Waiting Time']
    mean_special_customer_queue_waiting_time = data['Cumulative Stats']['Special Queue Waiting Time'] / len(
        data['Special Queue Customers All'])
    mean_normal_customer_queue_waiting_time = data['Cumulative Stats']['Normal Queue Waiting Time'] / len(
        data['Normal Queue Customers All'])

    mean_special_customer_technical_queue_waiting_time = data['Cumulative Stats'][
                                                             'Technical Special Queue Waiting Time'] / len(
        data['Technical Special Queue Customers All'])
    mean_normal_customer_technical_queue_waiting_time = data['Cumulative Stats'][
                                                            'Technical Normal Queue Waiting Time'] / len(
        data['Technical Normal Queue Customers All'])
    mean_special_customer_waiting_time_in_system = data['Special Customers Waiting Time In System'] / (
        len(data['Special Queue Customers All']))
    mean_normal_customer_waiting_time_in_system = data['Normal Customers Waiting Time In System'] / (
        len(data['Normal Queue Customers All']))
    #Yakhoda
    if data['Cumulative Stats']['Specialist Servers Last Time'] ==0:
        print('zero', clock)
        print(data['Cumulative Stats']['Beginner Servers Busy Time'])
    specialist_servers_utilization = data['Cumulative Stats']['Specialist Servers Busy Time'] / (num_of_specialists * data['Cumulative Stats']['Specialist Servers Last Time']) # ASAP_specialist #yakhoda
    beginner_servers_utilization = data['Cumulative Stats']['Beginner Servers Busy Time'] / (num_of_beginners * data['Cumulative Stats']['Beginner Servers Last Time']) # ASAP_beginner  #yakhoda         
    #debug
#     print(f'num_of_technicals is {num_of_technicals}')
#     print(data['Cumulative Stats']['Technical Servers Last Time'])
    technical_servers_utilization = data['Cumulative Stats']['Technical Servers Busy Time'] / (num_of_technicals * data['Cumulative Stats']['Technical Servers Last Time']) # ASAP_techinal #yakhoda
    #Yakhoda
    specialist_servers_overwork = data['Cumulative Stats']['Specialist Servers Last Time'] - work_time
    beginner_servers_overwork = data['Cumulative Stats']['Beginner Servers Last Time'] - work_time
    technical_servers_overwork = data['Cumulative Stats']['Technical Servers Last Time'] - work_time
    
    customer_churn_total.append(customer_churn)
    
    special_service_starters_total.append(special_service_starters)
    max_special_customer_queue_length_total.append(max_special_customer_queue_length)
    max_normal_customer_queue_length_total.append(max_normal_customer_queue_length)
    max_special_customer_technical_queue_length_total.append(max_special_customer_technical_queue_length)
    max_normal_customer_technical_queue_length_total.append(max_normal_customer_technical_queue_length)
    mean_special_customer_queue_length_total.append(mean_special_customer_queue_length)
    mean_normal_customer_queue_length_total.append(mean_normal_customer_queue_length)
    mean_special_customer_technical_queue_length_total.append(mean_special_customer_technical_queue_length)
    mean_normal_customer_technical_queue_length_total.append(mean_normal_customer_technical_queue_length)

    max_special_customer_queue_waiting_time_total.append(max_special_customer_queue_waiting_time)
    max_normal_customer_queue_length_waiting_time_total.append(max_normal_customer_queue_length_waiting_time)
    max_special_customer_technical_queue_waiting_time_total.append(max_special_customer_technical_queue_waiting_time)
    max_normal_customer_technical_queue_waiting_time_total.append(max_normal_customer_technical_queue_waiting_time)
    mean_special_customer_queue_waiting_time_total.append(mean_special_customer_queue_waiting_time)
    mean_normal_customer_queue_waiting_time_total.append(mean_normal_customer_queue_waiting_time)

    mean_special_customer_technical_queue_waiting_time_total.append(mean_special_customer_technical_queue_waiting_time)
    mean_normal_customer_technical_queue_waiting_time_total.append(mean_normal_customer_technical_queue_waiting_time)
    mean_special_customer_waiting_time_in_system_total.append(mean_special_customer_waiting_time_in_system)
    mean_normal_customer_waiting_time_in_system_total.append(mean_normal_customer_waiting_time_in_system)
    #Yakhoda
    specialist_servers_utilization_total.append(specialist_servers_utilization)
    beginner_servers_utilization_total.append(beginner_servers_utilization)
    technical_servers_utilization_total.append(technical_servers_utilization)
    #Yakhoda
    specialist_servers_overwork_total.append(specialist_servers_overwork)
    beginner_servers_overwork_total.append(beginner_servers_overwork)
    technical_servers_overwork_total.append(technical_servers_overwork)
    # if you want to have each replication's information, uncomment the code below.
    # print(data['Cumulative Stats']['Beginner Servers Busy Time'])
    # print(f'beginner_servers_utilization is {beginner_servers_utilization}')
    # print(f'customer_churn is {customer_churn}')
    # print(f'special_service_starters is {special_service_starters}')
    # print(f'max_special_customer_queue_length is {max_special_customer_queue_length}')
    # print(f'max_normal_customer_queue_length is {max_normal_customer_queue_length}')
    # print(f'max_special_customer_technical_queue_length is {max_special_customer_technical_queue_length}')
    # print(f'max_normal_customer_technical_queue_length is {max_normal_customer_technical_queue_length}')
    # print(f'mean_special_customer_queue_length is {mean_special_customer_queue_length}')
    # print(f'mean_normal_customer_queue_length is {mean_normal_customer_queue_length}')
    # print(f'mean_special_customer_technical_queue_length is {mean_special_customer_technical_queue_length}')
    # print(f'mean_normal_customer_technical_queue_length is {mean_normal_customer_technical_queue_length}')
    # print(f'max_special_customer_queue_waiting_time is {max_special_customer_queue_waiting_time}')
    # print(f'max_normal_customer_queue_length_waiting_time is {max_normal_customer_queue_length_waiting_time}')
    # print(f'max_special_customer_technical_queue_waiting_time is {max_special_customer_technical_queue_waiting_time}')
    # print(f'max_normal_customer_technical_queue_waiting_time is {max_normal_customer_technical_queue_waiting_time}')
    # print(f'mean_special_customer_queue_waiting_time is {mean_special_customer_queue_waiting_time}')
    # print(f'mean_normal_customer_queue_waiting_time is {mean_normal_customer_queue_waiting_time}')
    # print(f'mean_special_customer_technical_queue_waiting_time is {mean_special_customer_technical_queue_waiting_time}')
    # print(f'mean_normal_customer_technical_queue_waiting_time is {mean_normal_customer_technical_queue_waiting_time}')
    # print(f'specialist_servers_utilization is {specialist_servers_utilization}')
    # print(f'beginner_servers_utilization is {beginner_servers_utilization}')
    # print(f'technical_servers_utilization is {technical_servers_utilization}')
    # print(f'mean_special_customer_waiting_time_in_system is {mean_special_customer_waiting_time_in_system}')
    # print('_________________________________________')
    # print(f'{customer_churn}')
    # print(f'{special_service_starters}')
    # print(f'{max_special_customer_queue_length}')
    # print(f'{max_normal_customer_queue_length}')
    # print(f'{max_special_customer_technical_queue_length}')
    # print(f'{max_normal_customer_technical_queue_length}')
    # print(f'{mean_special_customer_queue_length}')
    # print(f'{mean_normal_customer_queue_length}')
    # print(f'{mean_special_customer_technical_queue_length}')
    # print(f'{mean_normal_customer_technical_queue_length}')
    # print(f'{max_special_customer_queue_waiting_time}')
    # print(f'{max_normal_customer_queue_length_waiting_time}')
    # print(f'{max_special_customer_technical_queue_waiting_time}')
    # print(f'{max_normal_customer_technical_queue_waiting_time}')
    # print(f'{mean_special_customer_queue_waiting_time}')
    # print(f'{mean_normal_customer_queue_waiting_time}')
    # print(f'{mean_special_customer_technical_queue_waiting_time}')
    # print(f'{mean_normal_customer_technical_queue_waiting_time}')
    # print(f'{specialist_servers_utilization}')
    # print(f'{beginner_servers_utilization}')
    # print(f'{technical_servers_utilization}')
    # print(f'{mean_special_customer_waiting_time_in_system}')
    # print('_________________________________________')

    return data
    # print(data)
# simulation(200*1440)




# if you wanted to run the program so many times and take an average for each result, you may uncomment the bottom code.
for i in range(1):
    simulation(8*60)

# print('customer_churn_total ', sum(customer_churn_total) / len(customer_churn_total))
# print('special_service_starters_total ',sum(special_service_starters_total) / len(special_service_starters_total))
# print('max_special_customer_queue_length_total ',sum(max_special_customer_queue_length_total) / len(max_special_customer_queue_length_total))
# print('max_normal_customer_queue_length_total ',sum(max_normal_customer_queue_length_total) / len(max_normal_customer_queue_length_total))
# print('max_special_customer_technical_queue_length_total ',sum(max_special_customer_technical_queue_length_total) / len(max_special_customer_technical_queue_length_total))
# print('max_normal_customer_technical_queue_length_total ',sum(max_normal_customer_technical_queue_length_total) / len(max_normal_customer_technical_queue_length_total))
# print('mean_special_customer_queue_length_total ',sum(mean_special_customer_queue_length_total) / len(mean_special_customer_queue_length_total))
# print('mean_normal_customer_queue_length_total ',sum(mean_normal_customer_queue_length_total) / len(mean_normal_customer_queue_length_total))
# print('mean_special_customer_technical_queue_length_total ',sum(mean_special_customer_technical_queue_length_total) / len(mean_special_customer_technical_queue_length_total))
# print('mean_normal_customer_technical_queue_length_total ',sum(mean_normal_customer_technical_queue_length_total) / len(mean_normal_customer_technical_queue_length_total))
# print('max_special_customer_queue_waiting_time_total ',sum(max_special_customer_queue_waiting_time_total) / len(max_special_customer_queue_waiting_time_total))
# print('max_normal_customer_queue_length_waiting_time_total ',sum(max_normal_customer_queue_length_waiting_time_total) / len(max_normal_customer_queue_length_waiting_time_total))
# print('max_special_customer_technical_queue_waiting_time_total ',sum(max_special_customer_technical_queue_waiting_time_total) / len(max_special_customer_technical_queue_waiting_time_total))
# print('max_normal_customer_technical_queue_waiting_time_total ',sum(max_normal_customer_technical_queue_waiting_time_total) / len(max_normal_customer_technical_queue_waiting_time_total))
# print('mean_special_customer_queue_waiting_time_total ',sum(mean_special_customer_queue_waiting_time_total) / len(mean_special_customer_queue_waiting_time_total))
# print('mean_normal_customer_queue_waiting_time_total ',sum(mean_normal_customer_queue_waiting_time_total) / len(mean_normal_customer_queue_waiting_time_total))
# print('mean_special_customer_technical_queue_waiting_time_total ',sum(mean_special_customer_technical_queue_waiting_time_total) / len(mean_special_customer_technical_queue_waiting_time_total))
# print('mean_special_customer_waiting_time_in_system_total ',sum(mean_special_customer_waiting_time_in_system_total) / len(mean_special_customer_waiting_time_in_system_total))
# print('specialist_servers_utilization_total ',sum(specialist_servers_utilization_total) / len(specialist_servers_utilization_total))
# print('beginner_servers_utilization_total ',sum(beginner_servers_utilization_total) / len(beginner_servers_utilization_total))
# print('technical_servers_utilization_total ',sum(technical_servers_utilization_total) / len(technical_servers_utilization_total))







# mean_normal_customer_queue_waiting_time_total = np.array(mean_normal_customer_queue_waiting_time_total)
# mean_normal_customer_queue_length_total = np.array(mean_normal_customer_queue_length_total) 
# mean_special_customer_queue_waiting_time_total = np.array(mean_special_customer_queue_waiting_time_total)
# mean_special_customer_technical_queue_waiting_time_total = np.array(mean_special_customer_technical_queue_waiting_time_total)
# mean_normal_customer_technical_queue_length_total = np.array(mean_normal_customer_technical_queue_length_total)
# specialist_servers_utilization_total = np.array(specialist_servers_utilization_total)
# customer_churn_total = np.array(customer_churn_total)


# mean_special_customer_waiting_time_in_system_total = np.array(mean_special_customer_waiting_time_in_system_total)
# mean_normal_customer_waiting_time_in_system_total = np.array(mean_normal_customer_waiting_time_in_system_total)
# #Yakhoda
# mean_special_customer_queue_waiting_time_total = np.array(mean_special_customer_queue_waiting_time_total)
# mean_normal_customer_queue_waiting_time_total = np.array(mean_normal_customer_queue_waiting_time_total)
# mean_special_customer_technical_queue_length_total = np.array(mean_special_customer_technical_queue_length_total)
# mean_normal_customer_technical_queue_length_total = np.array(mean_normal_customer_technical_queue_length_total)
# specialist_servers_utilization_total = np.array(specialist_servers_utilization_total)
# beginner_servers_utilization_total = np.array(beginner_servers_utilization_total)
# technical_servers_utilization_total = np.array(technical_servers_utilization_total)

# specialist_servers_overwork_total = np.array(specialist_servers_overwork_total)
# beginner_servers_overwork_total = np.array(beginner_servers_overwork_total)
# technical_servers_overwork_total = np.array(technical_servers_overwork_total)


# customer_churn_total = np.array(customer_churn_total)


print('-'*50)
print(np.mean(mean_special_customer_waiting_time_in_system_total))
print(np.std(mean_special_customer_waiting_time_in_system_total))


print('-'*50)
print(np.mean(mean_normal_customer_waiting_time_in_system_total))
print(np.std(mean_normal_customer_waiting_time_in_system_total))
#Yakhoda

# print('-'*50)
# print(np.mean(mean_special_customer_queue_waiting_time_total))
# print(np.std(mean_special_customer_queue_waiting_time_total))

# print('-'*50)
# print(np.mean(mean_normal_customer_queue_waiting_time_total))
# print(np.std(mean_normal_customer_queue_waiting_time_total))


print('-'*50)
print(np.mean(specialist_servers_utilization_total))
print(np.std(specialist_servers_utilization_total))

print('-'*50)
print(np.mean(beginner_servers_utilization_total))
print(np.std(beginner_servers_utilization_total))




print('-'*50)
print(np.mean(customer_churn_total))
print(np.std(customer_churn_total))

----------------------------------------------------------------------------------
Simulation Ended!
--------------------------------------------------
6.491151828662342
0.0
--------------------------------------------------
14.09778993527709
0.0
--------------------------------------------------
0.12477008007159286
0.0
--------------------------------------------------
0.1263998594527444
0.0
--------------------------------------------------
0.017608897126969416
0.0


In [192]:
# Since we have stored the state of 5000 days, we just use one day's stored states
np.savetxt('../../data/state_inputs_for_monitoring.txt', state_stored_for_monitoring)
np.savetxt('../../data/state_times_for_monitoring.txt', state_time_monitoring)

In [146]:
def truncate_lists_in_dict(input_dict):
    # Create a new dictionary to store the truncated results
    truncated_dict = {}
    
    # Iterate over the dictionary items
    for key, value in input_dict.items():
        # Check if the value is a list and its length exceeds 100
        if isinstance(value, list) and len(value) > 100:
            # Truncate the list to its first 100 elements
            truncated_dict[key] = value[:100]
        else:
            # Otherwise, keep the original list
            truncated_dict[key] = value
    
    return truncated_dict


In [147]:
state_stored_dict = dict()
state_stored = list()
for i in range(5000):
    print(i)
    

    
    simulation(8*60)
    
    
    

0
----------------------------------------------------------------------------------
Simulation Ended!
1
----------------------------------------------------------------------------------
Simulation Ended!
2
----------------------------------------------------------------------------------
Simulation Ended!
3
----------------------------------------------------------------------------------
Simulation Ended!
4
----------------------------------------------------------------------------------
Simulation Ended!
5
----------------------------------------------------------------------------------
Simulation Ended!
6
----------------------------------------------------------------------------------
Simulation Ended!
7
----------------------------------------------------------------------------------
Simulation Ended!
8
----------------------------------------------------------------------------------
Simulation Ended!
9
-----------------------------------------------------------------------

----------------------------------------------------------------------------------
Simulation Ended!
84
----------------------------------------------------------------------------------
Simulation Ended!
85
----------------------------------------------------------------------------------
Simulation Ended!
86
----------------------------------------------------------------------------------
Simulation Ended!
87
----------------------------------------------------------------------------------
Simulation Ended!
88
----------------------------------------------------------------------------------
Simulation Ended!
89
----------------------------------------------------------------------------------
Simulation Ended!
90
----------------------------------------------------------------------------------
Simulation Ended!
91
----------------------------------------------------------------------------------
Simulation Ended!
92
----------------------------------------------------------------

----------------------------------------------------------------------------------
Simulation Ended!
166
----------------------------------------------------------------------------------
Simulation Ended!
167
----------------------------------------------------------------------------------
Simulation Ended!
168
----------------------------------------------------------------------------------
Simulation Ended!
169
----------------------------------------------------------------------------------
Simulation Ended!
170
----------------------------------------------------------------------------------
Simulation Ended!
171
----------------------------------------------------------------------------------
Simulation Ended!
172
----------------------------------------------------------------------------------
Simulation Ended!
173
----------------------------------------------------------------------------------
Simulation Ended!
174
-------------------------------------------------------

----------------------------------------------------------------------------------
Simulation Ended!
246
----------------------------------------------------------------------------------
Simulation Ended!
247
----------------------------------------------------------------------------------
Simulation Ended!
248
----------------------------------------------------------------------------------
Simulation Ended!
249
----------------------------------------------------------------------------------
Simulation Ended!
250
----------------------------------------------------------------------------------
Simulation Ended!
251
----------------------------------------------------------------------------------
Simulation Ended!
252
----------------------------------------------------------------------------------
Simulation Ended!
253
----------------------------------------------------------------------------------
Simulation Ended!
254
-------------------------------------------------------

----------------------------------------------------------------------------------
Simulation Ended!
326
----------------------------------------------------------------------------------
Simulation Ended!
327
----------------------------------------------------------------------------------
Simulation Ended!
328
----------------------------------------------------------------------------------
Simulation Ended!
329
----------------------------------------------------------------------------------
Simulation Ended!
330
----------------------------------------------------------------------------------
Simulation Ended!
331
----------------------------------------------------------------------------------
Simulation Ended!
332
----------------------------------------------------------------------------------
Simulation Ended!
333
----------------------------------------------------------------------------------
Simulation Ended!
334
-------------------------------------------------------

...

----------------------------------------------------------------------------------
Simulation Ended!
4648
----------------------------------------------------------------------------------
Simulation Ended!
4649
----------------------------------------------------------------------------------
Simulation Ended!
4650
----------------------------------------------------------------------------------
Simulation Ended!
4651
----------------------------------------------------------------------------------
Simulation Ended!
4652
----------------------------------------------------------------------------------
Simulation Ended!
4653
----------------------------------------------------------------------------------
Simulation Ended!
4654
----------------------------------------------------------------------------------
Simulation Ended!
4655
----------------------------------------------------------------------------------
Simulation Ended!
4656
----------------------------------------------

----------------------------------------------------------------------------------
Simulation Ended!
4726
----------------------------------------------------------------------------------
Simulation Ended!
4727
----------------------------------------------------------------------------------
Simulation Ended!
4728
----------------------------------------------------------------------------------
Simulation Ended!
4729
----------------------------------------------------------------------------------
Simulation Ended!
4730
----------------------------------------------------------------------------------
Simulation Ended!
4731
----------------------------------------------------------------------------------
Simulation Ended!
4732
----------------------------------------------------------------------------------
Simulation Ended!
4733
----------------------------------------------------------------------------------
Simulation Ended!
4734
----------------------------------------------

----------------------------------------------------------------------------------
Simulation Ended!
4808
----------------------------------------------------------------------------------
Simulation Ended!
4809
----------------------------------------------------------------------------------
Simulation Ended!
4810
----------------------------------------------------------------------------------
Simulation Ended!
4811
----------------------------------------------------------------------------------
Simulation Ended!
4812
----------------------------------------------------------------------------------
Simulation Ended!
4813
----------------------------------------------------------------------------------
Simulation Ended!
4814
----------------------------------------------------------------------------------
Simulation Ended!
4815
----------------------------------------------------------------------------------
Simulation Ended!
4816
----------------------------------------------

----------------------------------------------------------------------------------
Simulation Ended!
4889
----------------------------------------------------------------------------------
Simulation Ended!
4890
----------------------------------------------------------------------------------
Simulation Ended!
4891
----------------------------------------------------------------------------------
Simulation Ended!
4892
----------------------------------------------------------------------------------
Simulation Ended!
4893
----------------------------------------------------------------------------------
Simulation Ended!
4894
----------------------------------------------------------------------------------
Simulation Ended!
4895
----------------------------------------------------------------------------------
Simulation Ended!
4896
----------------------------------------------------------------------------------
Simulation Ended!
4897
----------------------------------------------

----------------------------------------------------------------------------------
Simulation Ended!
4967
----------------------------------------------------------------------------------
Simulation Ended!
4968
----------------------------------------------------------------------------------
Simulation Ended!
4969
----------------------------------------------------------------------------------
Simulation Ended!
4970
----------------------------------------------------------------------------------
Simulation Ended!
4971
----------------------------------------------------------------------------------
Simulation Ended!
4972
----------------------------------------------------------------------------------
Simulation Ended!
4973
----------------------------------------------------------------------------------
Simulation Ended!
4974
----------------------------------------------------------------------------------
Simulation Ended!
4975
----------------------------------------------

Some other experiments

In [148]:
# state_stored_dict[(0,0,0,0)]

In [177]:
from collections import Counter

# Sample list of 4D tuples
tuples_list = state_stored

# Count the frequency of each tuple
tuple_counter = Counter(tuples_list)

# Convert Counter object to a list of unique tuples and their frequencies
unique_tuples_with_frequency = [(tuple_, count) for tuple_, count in tuple_counter.items()]

print(len(unique_tuples_with_frequency))


26417


In [178]:
from collections import Counter

def frequency_of_frequencies(tuples_list):
    # Count the frequency of each tuple
    tuple_counter = Counter(tuples_list)

    # Get the frequencies
    frequencies = tuple_counter.values()

    # Count the frequency of each frequency
    frequency_counter = Counter(frequencies)

    # Sort the frequency of frequencies by frequency
    sorted_frequency_of_frequencies = sorted(frequency_counter.items())

    return sorted_frequency_of_frequencies

# Sample list of 4D tuples
tuples_list = state_stored

result = frequency_of_frequencies(tuples_list)



[(1, 21929), (2, 3001), (3, 828), (4, 268), (5, 110), (6, 73), (7, 36), (8, 39), (9, 20), (10, 30), (11, 17), (12, 11), (13, 6), (14, 5), (15, 4), (16, 4), (17, 7), (18, 3), (19, 3), (20, 2), (21, 1), (22, 4), (23, 2), (24, 2), (25, 2), (26, 6), (27, 1), (29, 2), (126, 1)]


In [179]:
def filter_dict_by_list_length(input_dict, min_length):
    return {key: value for key, value in input_dict.items() if len(value) >= min_length}

# Sample dictionary
my_dict = state_stored_dict
# Minimum length of the list
min_length = 10

# Create a new dictionary containing only the key-value pairs where the list length is greater than min_length
filtered_dict = filter_dict_by_list_length(my_dict, min_length)

print(len(filtered_dict))


113


In [180]:
keys_list = list(filtered_dict)
values_list = list(filtered_dict.values())

In [184]:
# Here we determine which outputs we want to send to the clustering algorithm. 
# values_list = [[ point[3:6] + point[6:7]  for point in inner_list] for inner_list in values_list]

values_list = [[  tuple([np.abs((point[4] + point[5] + point[7])/3)]) + tuple([np.abs(point[2] + point[3])]) + tuple([np.abs(point[6])])  for point in inner_list] for inner_list in values_list]

In [187]:
for i in range(len(keys_list)):
    my_key = keys_list[i]
    my_value = values_list[i]
    if len(my_value)>100:
        values_list[i] = values_list[i][:100]

In [188]:
with open('../../data/state_outputs.txt', 'w') as f:
    # Write each element of the list to the file
    for item in values_list:
        f.write('%s\n' % item)

In [189]:
np.savetxt('../../data/state_inputs.txt', keys_list)


